In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster, HeatMap

In [73]:
df = pd.read_csv('Dados/Projecto 3_ Análise Espacial de Acidentes de Viação na Área Metropolitana de Maputo/dataset_acidentes.csv', encoding='latin1')
df.head(3)

,id,data,hora,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,condicao_metereologica,tracado_via,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude
0,262002,1/10/2020,21:40:00,Falta de Atenção à Condução,Colisão traseira,Com Vítimas Feridas,Plena Noite,Céu Claro,Reta,2,0,1,0,1,0,1,2,-25.947978,32.565075
1,264173,1/21/2020,8:05:00,Falta de Atenção à Condução,Colisão traseira,Com Vítimas Feridas,Pleno dia,Céu Claro,Reta,2,0,1,0,1,0,1,2,-25.964917,32.380069
2,270555,2/20/2020,21:40:00,Falta de Atenção à Condução,Colisão traseira,Com Vítimas Feridas,Plena Noite,Céu Claro,Reta,2,0,1,0,1,0,1,2,-25.872439,32.610327


In [79]:
df['geometry'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis = 1)
df.head(2)

,id,data,hora,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,condicao_metereologica,tracado_via,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,geometry
0,262002,1/10/2020,21:40:00,Falta de Atenção à Condução,Colisão traseira,Com Vítimas Feridas,Plena Noite,Céu Claro,Reta,2,0,1,0,1,0,1,2,-25.947978,32.565075,POINT (32.56507534 -25.94797822)
1,264173,1/21/2020,8:05:00,Falta de Atenção à Condução,Colisão traseira,Com Vítimas Feridas,Pleno dia,Céu Claro,Reta,2,0,1,0,1,0,1,2,-25.964917,32.380069,POINT (32.38006879 -25.96491672)


In [80]:
gdf = gpd.GeoDataFrame(df, geometry = 'geometry', crs = 'EPSG: 4326')
gdf.to_file('Dados/Projecto 3_ Análise Espacial de Acidentes de Viação na Área Metropolitana de Maputo/acidentes.geojson', driver = 'GeoJSON')

In [88]:
def carregar():
    # Importando o dataset de acidentes
    gdf = gpd.read_file('Dados/Projecto 3_ Análise Espacial de Acidentes de Viação na Área Metropolitana de Maputo/acidentes.geojson')
    gdf['data_hora'] = gdf['data'] + ' ' + gdf['hora']

    classe = gdf['classificacao_acidente'].unique()
    def classAcidentes(n):
        gdf_filtrado = gdf[gdf['classificacao_acidente'] == classe[n]]
        return gdf_filtrado
    
    gdf_semVitimas = classAcidentes(1)
    gdf_vitimasFeridas = classAcidentes(0)
    gdf_vitimasFatais = classAcidentes(2)
    return gdf, gdf_semVitimas, gdf_vitimasFeridas, gdf_vitimasFatais

gdf_acidentes, gdf_semVitimas, gdf_vitimasFeridas, gdf_vitimasFatais = carregar()

In [ ]:
# Coordenadas médias para centralizar o mapa na área de estudo
latMean = gdf_acidentes['latitude'].mean()
longMean = gdf_acidentes['longitude'].mean()

# Criação do mapa folium
map = folium.Map(location = [latMean, longMean], zoom_start = 10)

# Satélite
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr = 'Esri World Imagery',
    name = 'Satellite',
    show = False,
    overlay = False,
    control = True
).add_to(map)

# Coordenadas dos pontos de acorrência dos acidentes
def coordenadas(gdf):
    data = gdf[['latitude', 'longitude']].dropna().values.tolist()
    return data

# Criação dos pontos por severidade
def points(gdf, name, cor):
    return folium.GeoJson(gdf,
                name = name,
                marker = folium.Circle(radius = 50,
                                        fill_color = cor, 
                                        fill_opacity = 0.4,
                                        color = 'black',
                                        weight = 0.1
                            ),
                tooltip = folium.GeoJsonTooltip(fields = ['causa_acidente',
                                                            'tipo_acidente',
                                                            'data_hora',
                                                            'fase_dia',
                                                            'condicao_metereologica',
                                                            'mortos',
                                                            'feridos_leves',
                                                            'feridos_graves',
                                                            'ilesos',
                                                            'veiculos'
                                                            ],
                                                aliases = ['Estado:',
                                                            'Tipo:',
                                                            'Data:',
                                                            'Fase do Dia:',
                                                            'Condição Metereológica:',
                                                            'Mortos:',
                                                            'Feridos Leves:',
                                                            'Feridos Graves:',
                                                            'Ilesos:',
                                                            'Veículos Involvidos:'
                                                            ],
                                                    style = """
                                                                background-color: #F0EFEF;
                                                                border: 1px solid black;
                                                                border-radius: 5px;
                                                                box-shadow: 3px;
                                                            """
                            )
                        
    ).add_to(map)

points(gdf_semVitimas, 'Sem Vítimas 🟢', 'green')
points(gdf_vitimasFeridas, 'Vítimas Feridas 🟠', 'orange')
points(gdf_vitimasFatais, 'Vítimas Fatais 🔴', 'red')

HeatMap(coordenadas(gdf_acidentes),
                    name = 'HeatMap',
                    show = False
).add_to(map)

folium.LayerControl().add_to(map)

map.save('mapa.html')


In [1]:
gdf_acidentes['causa_acidente'].count()

NameError: name 'gdf_acidentes' is not defined